<a href="https://colab.research.google.com/github/Mario-RJunior/web-scraping-ML/blob/master/scraping_Mercado_Livre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import re
import numpy as np

In [2]:
re_precos = '<span class="price__fraction">(.*)</span>'
re_quartos = r'\| (.*) quarto'
re_areas = '> (.*) m² '
re_quarto_area_ausente = "> (.*) quarto"

In [3]:
zonas = ['norte', 'sul', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
dados = {'zona': [],
         'quartos': [],
         'area': [],
         'preco': []}

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'

    c = requests.get(url).content
    soup = BeautifulSoup(c, 'html.parser')

    preco = soup.find_all(name='span', attrs={'class': 'price__fraction'})
    atributos = soup.find_all(name='div', attrs={'class': 'item__attrs'})

    for p in preco:
      valor = re.findall(re_precos, str(p))[0]
      dados['preco'].append(valor)
    
    for atributo in atributos:
      n_quartos = re.findall(re_quartos, str(atributo))
      area = re.findall(re_areas, str(atributo))

      if n_quartos == []:
        n_quartos = re.findall(re_quarto_area_ausente, str(atributo))
        if n_quartos == []:
          n_quartos = np.nan
        else:
          n_quartos = n_quartos[0]
      else:
        n_quartos = n_quartos[0]
      dados['quartos'].append(n_quartos)

      if area == []:
        area = np.nan
      else:
        area = area[0]      
      dados['area'].append(area)

      dados['zona'].append(zona)
    sleep(5)

In [4]:
# Gerando do DataFrame
df = pd.DataFrame(dados)

In [5]:
# Visualizando o DataFrame
df

,zona,quartos,area,preco
0,norte,1,40,1.120
1,norte,2,47,1.414
2,norte,2,47,1.310
3,norte,2,51,1.123
4,norte,3,62,1.480
...,...,...,...,...
1147,oeste,1,35,1.850
1148,oeste,3,110,2.000
1149,oeste,3,192,7.000
1150,oeste,3,200,4.620


In [6]:
# Salvando o DataFrame em um arquivo .csv
df.to_csv('dados_imoveis.csv')